In [58]:
import cv2 as cv
import matplotlib.pyplot as plt
import timeit
import numpy as np
import random

print(cv.getVersionMajor())
print(cv.getVersionMinor())

# cfg_file = '/Users/patricklo/src/mypython_playground/yolo/yolov4-tiny.cfg'
# weight_file = '/Users/patricklo/src/mypython_playground/yolo/yolov4-tiny.weights'
# objname_file = '/Users/patricklo/src/mypython_playground/yolo/coco.names'

# mydnn = cv.dnn.readNetFromDarknet(cfg_file, weight_file)

# mydnn.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
# mydnn.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)
# wanda = plt.imread('/Users/patricklo/Downloads/download.jpeg')
# plt.imshow(wanda)

4
8


In [50]:
blob = cv.dnn.blobFromImage(wanda,0.00392, (608,608), [0,0,0], 1, crop=False)
mydnn.setInput(blob)

layerNames_temp = mydnn.getUnconnectedOutLayers() #get the indices of the yolo output layer
layer_i_yolo = [i - 1 for i in layerNames_temp]
layerNames = [mydnn.getLayerNames()[i] for i in layer_i_yolo]
print(layerNames)

t1 = timeit.default_timer()
netoutput = mydnn.forward(layerNames)
t2 = timeit.default_timer()
print(t2-t1)

['yolo_30', 'yolo_37']
0.1337001250000185


In [51]:
objnames = []
with open(objname_file) as file:
    for line in file:
        objnames.append(line.rstrip())
print(objnames)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [52]:
boxes = []
print(len(netoutput))
for what in netoutput:
    # print(len(what)) each output 1083, 4332
    for sth in what:
        #with each output, the output will be 5 + class number
        if sth[4] > 0.01:
            box = [sth[0], sth[1], sth[2], sth[3]]
            print(box)
        for i in range(len(objnames)):
            if sth[i+5] > 0.5:
                box.append(sth[i+5])
                box.append(objnames[i])
                boxes.append(box)

2
[0.5982879, 0.02265382, 0.16364981, 0.053120382]
[0.6622159, 0.027157081, 0.24289149, 0.058721874]
[0.70484763, 0.026097262, 0.22607106, 0.059833795]
[0.8201302, 0.024639102, 0.15944925, 0.06782102]
[0.8731782, 0.030888997, 0.18188927, 0.07066579]
[0.9202891, 0.035628363, 0.14057493, 0.07001231]
[0.07179589, 0.07321401, 0.14470191, 0.13566037]
[0.13091712, 0.07462297, 0.12526803, 0.14268197]
[0.66470194, 0.079212755, 0.1889968, 0.095662706]
[0.70732355, 0.080942094, 0.18351702, 0.08146258]
[0.76665425, 0.08741688, 0.20464717, 0.10561892]
[0.7604662, 0.09191717, 0.36614695, 0.18568419]
[0.81812644, 0.086802185, 0.14534014, 0.11849312]
[0.8673718, 0.08411188, 0.14697306, 0.10213517]
[0.9257889, 0.06894563, 0.14731126, 0.11831325]
[0.28736725, 0.12919267, 0.15536913, 0.12539811]
[0.33023843, 0.12352867, 0.30683756, 0.18779014]
[0.40524578, 0.1248698, 0.3069682, 0.21068977]
[0.4622516, 0.11993707, 0.22528747, 0.22076873]
[0.49140966, 0.122951806, 0.26510373, 0.23604265]
[0.66521186, 0.12

In [53]:
#nms
indices = cv.dnn.NMSBoxes(list(item[0:4] for item in boxes),list(item[4] for item in boxes), 0.1, 0.1)

In [54]:
for what in indices:
    pt0 = (  int(  np.shape(wanda)[1] * (boxes[what][0] - boxes[what][2] / 2) ), \
             int(  np.shape(wanda)[0] * (boxes[what][1] - boxes[what][3] / 2) ) )
    pt1 = (  int(  np.shape(wanda)[1] * (boxes[what][0] + boxes[what][2] / 2) ), \
             int(  np.shape(wanda)[0] * (boxes[what][1] + boxes[what][3] / 2) ) )

    cv.rectangle(wanda, pt0, pt1, (255, 153, 255), 8)
    cv.putText(wanda, boxes[what][5].upper(), (pt0[0], pt0[1]-10),cv.FONT_HERSHEY_COMPLEX_SMALL,1,(255,153,255))
    plt.imshow(wanda)
    plt.imsave('/Users/patricklo/src/mypython_playground/yolo/me.jpg', wanda)

In [55]:
print(t2-t1)

0.1337001250000185
